Styling of Jupyter widgets

This notebook presents how to style Jupyter interactive widgets to build rich and *reactive* widget-based applications.

You can jump directly to these sections:

+ [Predefined styles](#Predefined-styles)
+ [The style attribute](#The-style-attribute)

## Predefined styles

If you wish the styling of widgets to make use of colors and styles defined by the environment (to be consistent with e.g. a notebook theme), many widgets enable choosing in a list of pre-defined styles.

For example, the `Button` widget has a `button_style` attribute that may take 5 different values:

 - `'primary'`
 - `'success'`
 - `'info'`
 - `'warning'`
 - `'danger'`

besides the default empty string ''.

In [ ]:
from ipywidgets import Button

Button(description='Danger Button', button_style='danger')

## The `style` attribute

While the `layout` attribute only exposes layout-related CSS properties for the top-level DOM element of widgets, the  
`style` attribute is used to expose non-layout related styling attributes of widgets.

However, the properties of the `style` attribute are specific to each widget type.

In [ ]:
b1 = Button(description='Custom color')
b1.style.button_color = 'lightgreen'
b1

You can get a list of the style attributes for a widget with the `keys` property.

In [ ]:
b1.style.keys

Just like the `layout` attribute, widget styles can be assigned to other widgets.

In [ ]:
b2 = Button()
b2.style = b1.style
b2

Widget styling attributes are specific to each widget type.

In [ ]:
from ipywidgets import IntSlider

s1 = IntSlider(description='Blue handle')
s1.style.handle_color = 'lightblue'
s1

Styles can be given when a widget is constructed, either as a specific Style instance or as a dictionary.

In [ ]:
b3 = Button(description='Styled button', style=dict(
    font_style='italic',
    font_weight='bold',
    font_variant="small-caps",
    text_color='red',
    text_decoration='underline'
))
b3